In [ ]:
#| default_exp 05_map-amazon-meta-from-dump

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import json, gzip, argparse, os, pickle, numpy as np, uuid, ssl, scipy.sparse as sp
from timeit import default_timer as timer
from fastdownload import download_url
from multiprocessing import Pool
from functools import partial
from tqdm.auto import tqdm
from pathlib import Path
from PIL import Image

from sugar.core import *

ssl._create_default_https_context = ssl._create_unverified_context

## Load data

In [ ]:
#| export
def default_condition(o): 
    return True

def condition_a23(o, ids, key):
    return o[key] in ids

def get_condition(data_dir, dtype, key):
    all_ids = get_all_ids(f'{data_dir}/raw_data', encoding='latin-1')
    
    if dtype == 'a23': 
        return partial(condition_a23, ids=all_ids, key=key)
    elif dtype is None:
        return default_condition
    else: 
        raise ValueError(f'Invalid condition: {dtype}')
        

In [ ]:
#| export
def read_jsongz(fname, condition=default_condition):
    with gzip.open(fname, 'rt', encoding='utf-8') as file:
        return [json.loads(o) for o in file if condition(json.loads(o))]

def read_jsonl(fname, condition=default_condition):
    with open(fname, 'r') as file:
        return [json.loads(o) for o in file if condition(json.loads(o))]
        

In [ ]:
#| export
def read_file(fname, condition=default_condition):
    if fname.endswith('.jsonl'):
        return read_jsonl(fname, condition)
    elif fname.endswith('.json.gz'):
        return read_jsongz(fname, condition)
    else:
        raise ValueError(f'Invalid file: {fname}')
        

In [ ]:
fname = '/home/scai/phd/aiz218323/scratch/datasets/amazon/dumps/raw/meta_categories/meta_All_Beauty.jsonl'
items = read_file(fname)

In [ ]:
#| export
def get_items(cache_dir, condition=default_condition):
    items = []
    files = [f'{cache_dir}/{fname}' for fname in os.listdir(cache_dir) if fname.endswith('.jsonl') or fname.endswith('.json.gz')]
    for file in tqdm(files): items.extend(read_file(file, condition))
    return items
    

In [ ]:
cache_dir = '/home/scai/phd/aiz218323/scratch/datasets/amazon/dumps/raw/meta_categories/'
items = get_items(cache_dir)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#| export
def load_items(cache_dir:str, data_dir:str, key:str, condition_type=None):
    cache_file = f'{cache_dir}/products.pkl' if condition_type is None else f'{cache_dir}/products_{condition_type}.pkl'

    try:
        with open(cache_file, 'rb') as file:
            items = pickle.load(file)
    except:
        is_valid = get_condition(data_dir, condition_type, key)
        items = get_items(cache_dir, condition=is_valid)
        
        with open(cache_file, 'wb') as file:
            pickle.dump(items, file)

    return items
    

In [ ]:
cache_dir = '/home/scai/phd/aiz218323/scratch/datasets/amazon/dumps/raw/meta_categories/'
data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/LF-Amazon-131K/'
items = load_items(cache_dir, data_dir, key='parent_asin', condition_type='a23')

In [ ]:
items[3]['images']

[{'thumb': 'https://m.media-amazon.com/images/I/516glRKO6oL._SX38_SY50_CR,0,0,38,50_.jpg',
  'large': 'https://m.media-amazon.com/images/I/516glRKO6oL.jpg',
  'variant': 'MAIN',
  'hi_res': 'https://m.media-amazon.com/images/I/71mjUMNYXrL._SL1500_.jpg'},
 {'thumb': 'https://m.media-amazon.com/images/I/41HdbnZfhsL._SX38_SY50_CR,0,0,38,50_.jpg',
  'large': 'https://m.media-amazon.com/images/I/41HdbnZfhsL.jpg',
  'variant': 'PT01',
  'hi_res': 'https://m.media-amazon.com/images/I/61Ikt4caK7L._SL1500_.jpg'},
 {'thumb': 'https://m.media-amazon.com/images/I/41KJfqyZpDL._SX38_SY50_CR,0,0,38,50_.jpg',
  'large': 'https://m.media-amazon.com/images/I/41KJfqyZpDL.jpg',
  'variant': 'DTLS',
  'hi_res': 'https://m.media-amazon.com/images/I/81i4uKq5hvL._SL1500_.jpg'}]

In [ ]:
items[3]['videos']

[{'title': 'Tazo Tea Passion, The Only Tea I Have Ever Enjoyed',
  'url': 'https://www.amazon.com/vdp/0e4f45b1b94c471588e0822f5931d28b?ref=dp_vse_rvc_0',
  'user_id': '/shop/sethgaffar'},
 {'title': 'Tazo Refreshers Herbal Tea & Juice Mix Watermelon Cucumber',
  'url': 'https://www.amazon.com/vdp/0228aed725eb4936a9d3e88ebf136ba5?ref=dp_vse_rvc_1',
  'user_id': ''},
 {'title': 'Tazo Unsweetened Iced Herbal Tea',
  'url': 'https://www.amazon.com/vdp/0bd433867eed4e6b8f75e159c2bd6e35?ref=dp_vse_rvc_2',
  'user_id': ''}]

In [ ]:
cache_dir = '/home/scai/phd/aiz218323/scratch/datasets/amazon/dumps/raw/meta_categories/'
data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/(mapped)LF-AmazonTitles-1.3M/'

In [ ]:
items = load_items(cache_dir, data_dir, key='parent_asin', condition_type='a23')

  0%|          | 0/34 [00:00<?, ?it/s]

## Extract metadata

In [ ]:
#| export
def title_proc(o): 
    return [o['title']]
    
def description_proc(o): 
    return o['description']
    
def details_proc(o): 
    return [o['details'].__str__()]

def image_proc(o):
    images = list()
    for image in o['images']:
        for dtype in ['large', 'hi_res', 'thumb']:
            if dtype in image and image[dtype] is not None:
                images.append(image[dtype]); break
    assert len(images) == len(o['images']), f"Image not found: {o['title']}"
    return images

def video_proc(o):
    return [video['url'] for video in o['videos'] if video['url'] is not None]
    

In [ ]:
video_proc(items[3])

['https://www.amazon.com/vdp/0e4f45b1b94c471588e0822f5931d28b?ref=dp_vse_rvc_0',
 'https://www.amazon.com/vdp/0228aed725eb4936a9d3e88ebf136ba5?ref=dp_vse_rvc_1',
 'https://www.amazon.com/vdp/0bd433867eed4e6b8f75e159c2bd6e35?ref=dp_vse_rvc_2']

In [ ]:
#| export
METADATA_PROCS = {
    'title': title_proc, 
    'details': details_proc, 
    'images': image_proc,
    'videos': video_proc,
    'description': description_proc, 
}

In [ ]:
#| export
def get_meta_proc(dtype):
    assert dtype in METADATA_PROCS, f'Invalid metadata processing function: {dtype}.'
    return METADATA_PROCS[dtype]

def extract_meta_info(items, dtype, key):
    func = get_meta_proc(dtype)
    return {o[key]: func(o) for o in items}
    

In [ ]:
meta_mapping = extract_meta_info(items, 'videos', 'parent_asin')

In [ ]:
#| export
def get_vocabulary(mapping):
    vocab, mapping_item2idx = create_vocab_and_item2idx(mapping)

    vocab_txt = sorted(vocab, key=lambda x: vocab[x])
    vocab_ids = list(range(len(vocab_txt)))

    return vocab_ids, vocab_txt, mapping_item2idx
    

In [ ]:
#| export
def get_ids(data_dir):
    trn_ids, _ = load_raw_txt(f'{data_dir}/raw_data/train.raw.txt', encoding='latin-1')
    tst_ids, _ = load_raw_txt(f'{data_dir}/raw_data/test.raw.txt', encoding='latin-1')
    lbl_ids, _ = load_raw_txt(f'{data_dir}/raw_data/label.raw.txt', encoding='latin-1')
    return trn_ids, tst_ids, lbl_ids
    

In [ ]:
#| export
def get_matrix(mapping_item2idx, vocab_size, trn_ids, tst_ids, lbl_ids):
    trn_mat, trn_ids = get_matrix_from_item2idx(mapping_item2idx, vocab_size, trn_ids)
    tst_mat, tst_ids = get_matrix_from_item2idx(mapping_item2idx, vocab_size, tst_ids)
    lbl_mat, lbl_ids = get_matrix_from_item2idx(mapping_item2idx, vocab_size, lbl_ids)
    return trn_mat, tst_mat, lbl_mat
    

In [ ]:
#| export
def filter_vocab(vocab_ids, vocab_txt, trn_mat, tst_mat, lbl_mat=None):
    valid_idx = np.where(trn_mat.getnnz(axis=0) > 0)[0]
    if lbl_mat is not None:
        lbl_idx = np.where(lbl_mat.getnnz(axis=0) > 0)[0]
        valid_idx = np.union1d(valid_idx, lbl_idx)

    trn_mat = trn_mat[:, valid_idx]
    tst_mat = tst_mat[:, valid_idx]
    if lbl_mat is not None: 
        lbl_mat = lbl_mat[:, valid_idx]
    
    vocab_ids, vocab_txt = [vocab_ids[i] for i in valid_idx], [vocab_txt[i] for i in valid_idx]

    return vocab_ids, vocab_txt, trn_mat, tst_mat, lbl_mat
    

In [ ]:
metadata_ids, metadata_txt, mapping_item2idx = get_vocabulary(meta_mapping)

  0%|          | 0/5622 [00:00<?, ?it/s]

## Download image

In [ ]:
#| export
def get_downloaded_image_filename(dest, name, suffix):
    start_index = 1
    candidate_name = name

    while (dest/f"{candidate_name}{suffix}").is_file():
        candidate_name = f"{candidate_name}{start_index}"
        start_index += 1

    return candidate_name
    

In [ ]:
#| export
def download_image(dest, inp, timeout=4, preserve_filename=False):
    i,url = inp
    url = url.split("?")[0]
    url_path = Path(url)
    suffix = url_path.suffix if url_path.suffix else '.jpg'
    name = get_downloaded_image_filename(dest, url_path.stem, suffix) if preserve_filename else str(uuid.uuid4())
    try: 
        dest = dest/f"{name}{suffix}"
        download_url(url, dest, show_progress=False, timeout=timeout)
        return dest
    except Exception as e: 
        f"Couldn't download {url}."
        return None
        

In [ ]:
#| export
def download_images(dest, urls=None, n_workers=8, timeout=4, preserve_filename=False):
    dest = Path(dest)
    dest.mkdir(exist_ok=True)
    with Pool(processes=n_workers) as pool:
        fnames = list(tqdm(pool.imap(partial(download_image, dest, timeout=timeout, preserve_filename=preserve_filename), list(enumerate(urls))), total=len(urls)))
    return fnames
    

In [ ]:
#| export
def verify_image(fn):
    try:
        im = Image.open(fn)
        im.draft(im.mode, (32,32))
        im.load()
        return True
    except: return False

In [ ]:
#| export
def verify_images(fns, n_workers=8):
    with Pool(processes=n_workers) as pool:
        return list(tqdm(pool.imap(verify_image, fns), total=len(fns)))
        

In [ ]:
#| export
def remove_images(fnames, is_valid):
    for fname,v in zip(fnames, is_valid):
        if not v and fname is not None: fname.unlink()
    

In [ ]:
#| export
def filter_images(image_ids, image_txt, trn_mat, tst_mat, lbl_mat, valid_idx):
    trn_mat = trn_mat[:, valid_idx]
    tst_mat = tst_mat[:, valid_idx]
    lbl_mat = lbl_mat[:, valid_idx]
    
    image_ids, image_txt = [image_ids[i] for i in valid_idx], [image_txt[i] for i in valid_idx]

    return image_ids, image_txt, trn_mat, tst_mat, lbl_mat
    

In [ ]:
#| export
def process_images(metadata_ids, metadata_txt, trn_mat, tst_mat, lbl_mat):
    fnames = download_images(f'{data_dir}/images', urls=metadata_txt)
    
    is_valid = verify_images(fnames)
    remove_images(fnames, is_valid)

    metadata_txt = [o if o is None else o.name for o in fnames]
    return filter_images(metadata_ids, metadata_txt, trn_mat, tst_mat, lbl_mat, np.where(is_valid)[0])


In [ ]:
image_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/LF-Amazon-131K/images'

In [ ]:
fnames = download_images(image_dir, urls=metadata_txt)

  0%|          | 0/15722 [00:00<?, ?it/s]

In [ ]:
is_valid = verify_images(fnames)
remove_images(fnames, is_valid)

  0%|          | 0/15722 [00:00<?, ?it/s]

In [ ]:
metadata_txt = [o if o is None else o.name for o in fnames]

## Download video

In [ ]:
#| export
def get_downloaded_video_filename(dest, name, suffix):
    start_index = 1
    candidate_name = name

    while (dest/f"{candidate_name}{suffix}").is_file():
        candidate_name = f"{candidate_name}{start_index}"
        start_index += 1

    return candidate_name
    

In [ ]:
#| export
def download_video(dest, inp, timeout=4, preserve_filename=False):
    i,url = inp
    url = url.split("?")[0]
    url_path = Path(url)
    suffix = url_path.suffix if url_path.suffix else '.jpg'
    name = get_downloaded_image_filename(dest, url_path.stem, suffix) if preserve_filename else str(uuid.uuid4())
    try: 
        dest = dest/f"{name}{suffix}"
        download_url(url, dest, show_progress=False, timeout=timeout)
        return dest
    except Exception as e: 
        f"Couldn't download {url}."
        return None
        

In [ ]:
#| export
def download_videos(dest, urls=None, n_workers=8, timeout=4, preserve_filename=False):
    dest = Path(dest)
    dest.mkdir(exist_ok=True)
    with Pool(processes=n_workers) as pool:
        fnames = list(tqdm(pool.imap(partial(download_image, dest, timeout=timeout, preserve_filename=preserve_filename), list(enumerate(urls))), total=len(urls)))
    return fnames
    

In [ ]:
#| export
def verify_video(fn):
    try:
        im = Image.open(fn)
        im.draft(im.mode, (32,32))
        im.load()
        return True
    except: return False

In [ ]:
#| export
def verify_videos(fns, n_workers=8):
    with Pool(processes=n_workers) as pool:
        return list(tqdm(pool.imap(verify_image, fns), total=len(fns)))
        

In [ ]:
#| export
def remove_videos(fnames, is_valid):
    for fname,v in zip(fnames, is_valid):
        if not v and fname is not None: fname.unlink()
    

In [ ]:
#| export
def filter_videos(image_ids, image_txt, trn_mat, tst_mat, lbl_mat, valid_idx):
    trn_mat = trn_mat[:, valid_idx]
    tst_mat = tst_mat[:, valid_idx]
    lbl_mat = lbl_mat[:, valid_idx]
    
    image_ids, image_txt = [image_ids[i] for i in valid_idx], [image_txt[i] for i in valid_idx]

    return image_ids, image_txt, trn_mat, tst_mat, lbl_mat
    

In [ ]:
#| export
def process_videos(metadata_ids, metadata_txt, trn_mat, tst_mat, lbl_mat):
    fnames = download_images(f'{data_dir}/images', urls=metadata_txt)
    
    is_valid = verify_images(fnames)
    remove_images(fnames, is_valid)

    metadata_txt = [o if o is None else o.name for o in fnames]
    return filter_images(metadata_ids, metadata_txt, trn_mat, tst_mat, lbl_mat, np.where(is_valid)[0])


In [ ]:
video_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/LF-Amazon-131K/videos'

In [ ]:
metadata_txt[:5]

['https://www.amazon.com/vdp/0354b17a71a44097930f28168b8aee8f?ref=dp_vse_rvc_0',
 'https://www.amazon.com/vdp/0e4f45b1b94c471588e0822f5931d28b?ref=dp_vse_rvc_0',
 'https://www.amazon.com/vdp/0228aed725eb4936a9d3e88ebf136ba5?ref=dp_vse_rvc_1',
 'https://www.amazon.com/vdp/0bd433867eed4e6b8f75e159c2bd6e35?ref=dp_vse_rvc_2',
 'https://www.amazon.com/vdp/7615c3bde1d54f1abaeeff2ff13be5d8?ref=dp_vse_rvc_0']

In [ ]:
fnames = download_videos(video_dir, urls=metadata_txt)

  0%|          | 0/15722 [00:00<?, ?it/s]

In [ ]:
is_valid = verify_videos(fnames)
remove_videos(fnames, is_valid)

  0%|          | 0/15722 [00:00<?, ?it/s]

In [ ]:
metadata_txt = [o if o is None else o.name for o in fnames]

## Construct matrix

In [ ]:
#| export
def get_metadata(cache_dir, data_dir, meta_type, key, condition_type, do_filter=True):
    items = load_items(cache_dir, data_dir, key, condition_type)
    
    meta_mapping = extract_meta_info(items, meta_type, key)

    metadata_ids, metadata_txt, mapping_item2idx = get_vocabulary(meta_mapping)
    trn_ids, tst_ids, lbl_ids = get_ids(data_dir)
    trn_mat, tst_mat, lbl_mat = get_matrix(mapping_item2idx, len(metadata_ids), trn_ids, tst_ids, lbl_ids)

    if do_filter:
        metadata_ids, metadata_txt, trn_mat, tst_mat, lbl_mat = filter_vocab(metadata_ids, metadata_txt, trn_mat, tst_mat, lbl_mat)
        
    return trn_mat, tst_mat, lbl_mat, metadata_ids, metadata_txt
    

In [ ]:
#| export
def save_metadata(save_dir, trn_mat, tst_mat, lbl_mat, metadata_ids, metadata_txt, metadata_type):
    sp.save_npz(f'{save_dir}/{metadata_type}_trn_X_Y.npz', trn_mat)
    sp.save_npz(f'{save_dir}/{metadata_type}_tst_X_Y.npz', tst_mat)
    sp.save_npz(f'{save_dir}/{metadata_type}_lbl_X_Y.npz', lbl_mat)
    
    os.makedirs(f'{save_dir}/raw_data', exist_ok=True)
    save_raw_txt(f'{save_dir}/raw_data/{metadata_type}.raw.txt', metadata_ids, metadata_txt)
    

In [ ]:
cache_dir = '/home/scai/phd/aiz218323/scratch/datasets/amazon/dumps/raw/meta_categories/'
data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/LF-Amazon-131K/'

In [ ]:
trn_mat, tst_mat, lbl_mat, metadata_ids, metadata_txt = get_metadata(cache_dir, data_dir, meta_type='images', key='parent_asin', 
                                                                     condition_type='a23', do_filtering=True)

  0%|          | 0/5622 [00:00<?, ?it/s]

  0%|          | 0/294805 [00:00<?, ?it/s]

  0%|          | 0/134835 [00:00<?, ?it/s]

  0%|          | 0/131073 [00:00<?, ?it/s]

In [ ]:
save_metadata(data_dir, trn_mat, tst_mat, lbl_mat, metadata_ids, metadata_txt, metadata_type)

## `__main__`

In [ ]:
#| export
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--cache_dir', type=str, required=True)
    parser.add_argument('--data_dir', type=str, required=True)
    parser.add_argument('--key', type=str, default='parent_asin')
    parser.add_argument('--condition_type', type=str, default=None)
    parser.add_argument('--metadata_type', type=str, required=True)
    parser.add_argument('--no_filter', action='store_false')
    return parser.parse_args()


In [ ]:
#| export

if __name__ == '__main__':
    start_time = timer()

    args = parse_args()

    trn_mat, tst_mat, lbl_mat, metadata_ids, metadata_txt = get_metadata(args.cache_dir, args.data_dir, meta_type=args.metadata_type, 
                                                                         key=args.key, condition_type=args.condition_type, 
                                                                         do_filter=args.no_filter)
    save_metadata(args.data_dir, trn_mat, tst_mat, lbl_mat, metadata_ids, metadata_txt, args.metadata_type)
    
    end_time = timer()
    print(f'Time elapsed: {end_time-start_time:.2f} seconds.')
    